# GPU-Accelerated Numerical Computing with MatX

## Tutorial List
1. [Introduction](01_introduction.ipynb)
2. [Operators](02_operators.ipynb)
3. Executors (this tutorial)
4. [Radar Pipeline Example](04_radar_pipeline.ipynb)

## Executors
MatX executors are a generic name given to functions that execute work on the device. Operators and generators were introduced in the last tutorial as a way to generate a CUDA kernel from an expression, but they do not execute any work on the device. The `run` took an operator as input and executed it on the device. Many other types of executors exist in MatX where more complex functions can be executed alongside operators. Some executors are wrappers around existing CUDA libraries, while others are custom executors inside of MatX. This distinction is hidden from developers so that the implementation of an executor can change over time without modifying the client code. Some executors can take an operator as input, while others can only take tensors as input. These restrictions are noted in the MatX documentation, and may be relaxed or removed in future versions.

Besides `run`, other executors typically allow non-element-wise kernels to execute using highly-optimized library backends. Some examples of this would be a matrix multiply (GEMM), reduction, FFT, sorting, and linear solvers. Besides the type of inputs allowed, executors may also have restrictions on the rank and/or size of a tensor. For example, performing a GEMM requires that the tensors are at least rank 2 (i.e. be a matrix), and the last dimension of the first tensor must match the second-to-last dimension of the second tensor (`MxK * KxN`). Most executors support batching, and anything above the nominal rank will result in batching dimensions. In a 1D FFT this would mean that any dimension above 1 is treated as another 1D batched FFT, and a 2D FFT would batch any dimensions above 2. 

Some executors use CUDA libraries to implement their functionality, and those libraries require either a handle or a plan to operated. MatX hides this complexity by creating and caching the plan on the first call, and using the same plan on future calls where possible. More advanced users may use the handle interface directly to avoid the caching. Only the caching interface will be covered in this tutorial since it's the recommended approach, but the non-cached version can be found in the documentation.

In [1]:
//todo this should be moved to a hidden init block that runs automatically when the notebook starts
#pragma cling add_library_path("/usr/local/cuda/lib64")
#pragma cling add_library_path("/opt/xeus/cling/lib")
//#pragma cling add_library_path("/usr/Lib/gcc/x86_64-Linux-gnu/11/")
#pragma cling add_library_path("/usr/lib/x86_64-linux-gnu/openblas64-openmp/")
#pragma cling add_include_path("/usr/local/cuda/include")
#pragma cling add_include_path("/usr/include/x86_64-linux-gnu/openblas64-openmp")
#pragma cling add_include_path("/opt/xeus/cling/tools/Jupyter/kernel/MatX/include")
#pragma cling add_include_path("/opt/xeus/cling/tools/Jupyter/kernel/MatX/build/_deps/cccl-src/libcudacxx/include")
//#pragma cling load("libgomp")
#pragma cling load("libopenblas64")
#pragma cling load("libcuda")
#pragma cling load("libcudart")
#pragma cling load("libcurand")
#pragma cling load("libcublas")
#pragma cling load("libcublasLt")
#pragma cling load("libcufft")

#include <cuda/std/__algorithm/max.h>
#include <cuda/std/__algorithm/min.h>

#define MATX_EN_OPENBLAS
#define MATX_EN_OPENBLAS_LAPACK
#define MATX_OPENBLAS_64BITINT

#include "matx.h"

### Matrix Multiply
The `matmul` executor performs the matrix-matrix multiply of $$C = {\alpha}A * B + {\beta}C$$ where `A` is of dimensions `MxK`, `B` is `KxN`, and `C` is `MxN`. We first populate the `A` and `B` matrices with random values before the multiply as we did in the example above, then the GEMM is performed. Since the random number generator allocates memory sufficient to randomize the entire tensor, we create a random number generator large enough to generate values for both A or B. This allows us to create a single random number generator, but pull different random values for A and B by simply calling `run` twice. As mentioned above, any rank above 2 is consiered a batching dimension.

We use rectangular matrices for `A` and `B`, while `C` will be a square matrix due to the outer dimensions of `A` and `B` matching. 


Open the file [exercises/example3_gemm.cu](exercises/example3_gemm.cu) and edit the contents where you see TODO markers.

In [2]:
  auto A = matx::make_tensor<float>({8, 4});
  auto B = matx::make_tensor<float>({4, 8});
  auto C = matx::make_tensor<float>({8, 8});

  (A = matx::random<float>({8, 4}, matx::NORMAL)).run();  
  (B = matx::random<float>({4, 8}, matx::NORMAL)).run();  

  // TODO: Perform a GEMM of C = A*B
  (C = matx::matmul(A, B)).run();
  
  printf("A:\n");
  matx::print(A);
  printf("B:\n");
  matx::print(B);  
  printf("C:\n");
  matx::print(C);    


A:
tensor_2_f32: Tensor{float} Rank: 2, Sizes:[8, 4], Strides:[4,1]
000000:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000001:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000002:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000003:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000004:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000005:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000006:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000007:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
B:
tensor_2_f32: Tensor{float} Rank: 2, Sizes:[4, 8], Strides:[8,1]
000000:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000001:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000002:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000003:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  

(void) @0x78e507dfec30


### FFT
MatX provides an interface to do both 1D Fast Fourier Transforms (FFTs) and 2D FFTs. Any tensor above rank 1 will be batched in a 1D FFT, and any tensor above rank 2 will be batched in a 2D FFT. FFTs may either be done in-place or out-of-place by using the same or different variables for the output and inputs. Since the tensors are strongly-typed, the type of FFT (C2C, R2C, etc) is inferred by the tensor type at compile time. Similarly, the input and output size of the executor is deduced by the type of transform, and the input/output tensors must match those sizes. There's one exception to this rule, and it's when the input FFT is to be zero-padded at the end. In this case, the input tensor can be shorter than the output tensor, and the input will be zero-padded to the length of the output tensor. This is a common tactic used in signal and image processing for both speed and FFT resolution.

In this example, we execute a 1D batched FFT on a 2D tensor populated with random complex floating point data. Since the FFT executor is performed in-place, the input and output types of the tensors are the same, and the type of the FFT is inferred as a complex-to-complex (`C2C`). The FFT length is specified by the inner dimension of the tensor, or 4 in this example, and the outer dimension is the number of batches, or 2. After the FFT completes, we perform on IFFT on the same tensor using the `ifft` interface. Ignoring floating point inaccuracies, the result of `ifft(fft(A))` should be the same as `A`, and this is shown by printing the tensors at each step. To perform a batched FFT on columns instead of rows, the tensor can be transposed by calling the `Permute` function used in the first tutorial. When the library detects a permuted tensor is being used, it can use technique to speed the FFT up over the naive method of converting the data in memory.

In [3]:
auto D = matx::make_tensor<float>({2, 4});

// (D = matx::random<float>(D.Shape(), matx::NORMAL)).run();
// matx::print(D);

// (D = fft(D)).run();
// matx::print(D);

// (D = matx::ifft(D)).run();  
// matx::print(D);

Next, we take the same 2D tensor and perform a 2D FFT on it. Since the rank is 2, it will not be batched as in the previous example. 

In [4]:
(D = matx::random<float>(D.Shape(), matx::NORMAL)).run();
// matx::print(D);

// (D = fft2(D)).run();
// matx::print(D);

// (D = matx::ifft2(D)).run();  
// matx::print(D);

(void) @0x78e507dfec30


As before, the results after the IFFT closely match the original `C` tensor, but with floating point error.

### Reductions
A reduction operation takes multiple values and aggregates those into a smaller number of values. Most reductions take a large number of values and reduces them to a single value. Reductions are one of the most common operations perfomed on the GPU, which means they've been heavily researched and optimized for highly-parallel processors. Modern NVIDIA GPUs have special instructions for performing reductions to give even larger speedups over naive implementations. All of these details are hidden from the user and MatX automatically chooses the optimized path based on the hardware capabilities. 

MatX provides a set of optimized primitives to perform reductions on tensors for many common types. Reductions are supported across individual dimensions or on entire tensors, depending on the size of the output tensor. Currently supported reduction functions are `sum`, `min`, `max`,` mean`, `any`, and `all`.

#### Full Reduction
In this example we reduce an entire tensor to a single value by applying the reduction across all dimensions of the tensor. We apply the same random initialization from previous examples on a 2D tensor `A`. Note that the output tensor must be zeroed for a `sum` reduction since that value is continually added to during the reduction. Not initializing the output tensor will give undefined results since the variables are used as accumulators throughout the reduction. With the tensor initialized, we perform both a `max` and `sum` reduction across all dimensions of the tensor:


In [5]:

  auto MD0 = matx::make_tensor<float>({});
  auto AD0 = matx::make_tensor<float>({});

  (A = matx::random<float>(A.Shape(), matx::NORMAL)).run();    
  
  // Initialize max and average to 0
  (MD0 = 0).run();
  (AD0 = 0).run();

  (MD0 = max(A)).run();
  (AD0 = sum(A)).run();

  printf("A:\n");
  matx::print(A);
  printf("Max: %f\n", MD0());
  printf("Sum: %f\n", AD0()); 

A:
tensor_2_f32: Tensor{float} Rank: 2, Sizes:[8, 4], Strides:[4,1]
000000:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000001:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000002:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000003:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000004:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000005:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000006:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000007:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
Max: 0.000000
Sum: 0.000000


(int) 14


#### Dimensional Reductions
Reductions can also be performed across certain dimensions instead of the whole tensor. Dimensional reductions are useful in situations where each row contains data for a different user, for example, and we wish to sum up each user's data. By setting the output tensor view to a 1D tensor, independent reductions can be performed across the input tensor where each output element corresponds to a single row reduction from the input. Using the same tensor `A` from the previous example, we only change the output tensor type to be a 1D tensor instead of a scalar:


In [6]:

  auto MD1 = matx::make_tensor<float>({A.Size(0)});
  auto AD1 = matx::make_tensor<float>({A.Size(0)});

  (A = matx::random<float>(A.Shape(), matx::NORMAL)).run();    
  
  // Initialize max and average to 0
  (MD1 = 0).run();
  (AD1 = 0).run();

  (MD1 = max(A)).run();
  (AD1 = sum(A)).run();

(void) @0x78e507dfec30


Printing the new reduction tensors shows the reduced values across each row of the input tensor `A`.

In [7]:
  printf("A:\n");
  matx::print(A);
  printf("Max:\n");
  matx::print(MD1);
  printf("Sum:\n");
  matx::print(AD1);

A:
tensor_2_f32: Tensor{float} Rank: 2, Sizes:[8, 4], Strides:[4,1]
000000:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000001:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000002:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000003:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000004:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000005:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000006:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000007:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
Max:
tensor_1_f32: Tensor{float} Rank: 1, Sizes:[8], Strides:[1]
000000:  0.0000e+00 
000001:  0.0000e+00 
000002:  0.0000e+00 
000003:  0.0000e+00 
000004:  0.0000e+00 
000005:  0.0000e+00 
000006:  0.0000e+00 
000007:  0.0000e+00 
Sum:
tensor_1_f32: Tensor{float} Rank: 1, Sizes:[8], Strides:[1]
000000:  0.0000e+00 
000001:  0.0000e+00 
000002:  0.0000e+00 
000003:  0.0000e+00 
000004:  0.0000e+00 
000005:  0.0000e+00 
000006:  0.0000e+00 
000007:  0.0000e+00 


(void) @0x78e507dfec30


### Convolution
MatX supports both 1D and 2D direct convolution using the `conv1d` and `conv2d` functions. FFT-based convolution can also be performed as a combination of existing primitives as a potentially faster alternative to direct convolution for large tensors. Both forms of direct convolution take in an extra mode which specifies how much of the output is saved, where `MATX_C_MODE_FULL` saves the entire filter ramp-up and down, `MATX_C_MODE_SAME` makes the input and output tensors the same size, and `MATX_C_MODE_VALID` only keeps valid samples (when the entire filter was part of the convolution). Convolution can be used to perform a rolling average of an input by making all filter values 1/N, where N is the length of the filter. In this example, we use a filter of length 3 to create a running average of the last 3 elements:


In [8]:
auto CIn  = matx::make_tensor<float>({16});
auto filt = matx::make_tensor<float>({3});
auto Co   = matx::make_tensor<float>({16 + filt.Lsize() - 1});

filt.SetVals({1.0/3, 1.0/3, 1.0/3});

(CIn = matx::random<float>({16}, matx::NORMAL)).run();  

printf("Initial CIn tensor:\n");
matx::print(CIn);
(Co = matx::conv1d(CIn, filt, matx::MATX_C_MODE_FULL)).run();

matx::print(Co);

Initial CIn tensor:
tensor_1_f32: Tensor{float} Rank: 1, Sizes:[16], Strides:[1]
000000:  0.0000e+00 
000001:  0.0000e+00 
000002:  0.0000e+00 
000003:  0.0000e+00 
000004:  0.0000e+00 
000005:  0.0000e+00 
000006:  0.0000e+00 
000007:  0.0000e+00 
000008:  0.0000e+00 
000009:  0.0000e+00 
000010:  0.0000e+00 
000011:  0.0000e+00 
000012:  0.0000e+00 
000013:  0.0000e+00 
000014:  0.0000e+00 
000015:  0.0000e+00 
tensor_1_f32: Tensor{float} Rank: 1, Sizes:[18], Strides:[1]
000000:  0.0000e+00 
000001:  0.0000e+00 
000002:  0.0000e+00 
000003:  0.0000e+00 
000004:  0.0000e+00 
000005:  0.0000e+00 
000006:  0.0000e+00 
000007:  0.0000e+00 
000008:  0.0000e+00 
000009:  0.0000e+00 
000010:  0.0000e+00 
000011:  0.0000e+00 
000012:  0.0000e+00 
000013:  0.0000e+00 
000014:  0.0000e+00 
000015:  0.0000e+00 
000016:  0.0000e+00 
000017:  0.0000e+00 


(void) @0x78e507dfec30


Similar to a 1D convolution, a 2D convolution does the same computation over two dimensions. A tensor of at least rank 2 is needed for a 2D convolution. Below we use a filter of all ones using the `ones` operator to demonstrate the filter can also be an operator and not an existing tensor view. The result is the sum of the four values around each cell on the input:

In [9]:

  auto CIn2  = matx::make_tensor<float>({8,8});
  auto filt2 = matx::ones<float>({2, 2});
  auto Co2   = matx::make_tensor<float>({8, 8});

  (CIn2 = matx::random<float>({8, 8}, matx::NORMAL)).run();  

  printf("Initial C tensor:\n");
  matx::print(C);

  (Co2 = matx::conv2d(CIn2, filt, matx::MATX_C_MODE_SAME)).run();
  
  printf("After conv2d:\n");
  matx::print(Co2);



Initial C tensor:
tensor_2_f32: Tensor{float} Rank: 2, Sizes:[8, 8], Strides:[8,1]
000000:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000001:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000002:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000003:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000004:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000005:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000006:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
000007:  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 
After conv2d:
tensor_2_f32: Tensor{float} Rank: 2, Sizes:[8, 8], Strides:[8,1

(void) @0x78e507dfec30


Last, we mentioned above that convolution can also be done in the frequency domain using FFTs. This is the preferred method for larger tensors since FFTs are much faster than direct convolutions in large sizes, and because FFT libraries are highly-optimized. FFT convolution uses more memory than direct if the inputs are not to be destroyed since it requires running an FFT on both the input signal and filter before filtering. If not done in-place, this typically requires `2N + L - 1` new elements in memory, where N is the signal length and L is the filter length. A full FFT convolution example can be found in `fft_conv.cu` in the MatX examples, but the main convolution code is shown below:


In [10]:

///\todo complete this tutorial. This one is pretty complex, do we want to keep the validation code here?

// using complex = cuda::std::complex<float>;
// cudaExecutor exec{};

// index_t signal_size = 16;
// index_t filter_size = 3;
// index_t filtered_size = signal_size + filter_size - 1;

// // Create time domain buffers
// auto sig_time  = make_tensor<complex>({signal_size});
// auto filt_time = make_tensor<complex>({filter_size});
// auto time_out  = make_tensor<complex>({filtered_size});

// // Frequency domain buffers
// auto sig_freq  = make_tensor<complex>({filtered_size});
// auto filt_freq = make_tensor<complex>({filtered_size});

// // Fill the time domain signals with data
// for (index_t i = 0; i < signal_size; i++) {
//   sig_time(i) = {-1.0f * (2.0f * static_cast<float>(i % 2) + 1.0f) *
//                         (static_cast<float>(i % 10) / 10.0f) +
//                     0.1f,
//                 -1.0f * (static_cast<float>(i % 2) == 0.0f) *
//                         (static_cast<float>(i % 10) / 5.0f) -
//                     0.1f};
// }
// for (index_t i = 0; i < filter_size; i++) {
//   filt_time(i) = {static_cast<float>(i) / static_cast<float>(filter_size),
//                   static_cast<float>(-i) / static_cast<float>(filter_size) +
//                       0.5f};
// }

// TODO: Perform FFT convolution
// Perform the FFT in-place on both signal and filter
// (sig_freq = fft(sig_time)).run();
// (filt_freq = fft(filt_time)).run();

// (sig_freq = sig_freq * filt_freq).run();

// // IFFT in-place
// (sig_freq = ifft(sig_freq)).run();  


// Perform the FFT in-place on both signal and filter, do an element-wise multiply of the two, then IFFT that output
// (sig_freq = ifft(fft(sig_time, filtered_size) * fft(filt_time, filtered_size))).run(stream);

// TODO: Perform a time-domain convolution
// conv1d(time_out, sig_time, filt_time, matxConvCorrMode_t::MATX_C_MODE_FULL, 0);

// exec.sync();

// // Compare signals
// for (index_t i = 0; i < filtered_size; i++) {
//     if (  fabs(time_out(i).real() - sig_freq(i).real()) > 0.001 || 
//           fabs(time_out(i).imag() - sig_freq(i).imag()) > 0.001) {
//         printf("Verification failed at item %lld. Direct=%f%+.2fj, FFT=%f%+.2fj\n", i,
//           time_out(i).real(), time_out(i).imag(), sig_freq(i).real(), sig_freq(i).imag());
//         return -1;
//     }
// }

std::cout << "Verification successful" << std::endl;


Since the expected output size of the full filtering operation is signal_len + filter_len - 1, both the filter and signal time domain inputs are shorter than the output. This would normally require a separate stage of allocating buffers of the appropriate size, zeroing them out, copying the time domain data to the buffers, and performing the  FFT. However, MatX has an API to do all of this automatically in the library using asynchronous allocations. This makes the call have a noticeable performance hit on the first call, but subsequent calls will be close to the time without allocation. To recognize that automatic padding is wanted, MatX uses the output tensor size compared to the input tensor size to determine whether to pad the input with zeros. In this case the output signal (sig_time and filt_time) are shorter than the output tensors (sig_freq and filt_freq), so it will automatically zero-pad the input.

The above expression can also be combined into a single line:

```c++
(sig_freq = ifft(fft(sig_time, filtered_size) * fft(filt_time, filtered_size))).run(stream);
```

Using the convolution property $ h*x \leftrightarrow H \cdot X$ we simply multiply the signals element-wise after the FFT, then do an IFFT to go back to the time domain.

Next, we do the same operation in the time domain using the `conv1d` function:

```c++
conv1d(time_out, sig_time, filt_time, matxConvCorrMode_t::MATX_C_MODE_FULL, 0);
```

To match the FFT results we do a full convolution to get all the samples from the filter ramp up and ramp down. However, if we wanted either valid or same mode we could slice the FFT convolution output at the appropriate places to give the same answer. Edit the file [exercises/example3_fft_conv.cu](exercises/example3_fft_conv.cu) and add the missing code where you see TODOs. After running the verification code at the bottom will check for accuracy.

This concludes the third tutorial on MatX. In this tutorial you learned what executors are, and how they can be applied on tensor views. In the next example you will walk through an entire radar signal processing pipeline using all the primites learned up to this point. 

[Start Next Tutorial](04_radar_pipeline.ipynb)